<h1 align="center"><font color="red">RAG usando OpenAI, Weaviate e LangChain</font></h1>

<font color="yellow">Data Scientist.: Dr. Eddy Giusepe Chirinos Isidro</font>

![Fluxo de trabalho de geração aumentada de recuperação](https://miro.medium.com/v2/resize:fit:720/format:webp/1*kSkeaXRvRzbJ9SrFZaMoOg.png)

<h1 align="left"><font color="red">O que é geração aumentada de recuperação (RAG)?</font></h1>

`Retrieval-Augmented Generation` (RAG) é o conceito para fornecer aos LLMs informações adicionais de uma `fonte de conhecimento externa`. Isso lhes permite gerar respostas mais precisas e contextuais, ao mesmo tempo que reduz as `alucinações`.

In [ ]:
# Setup:
%pip install langchain openai weaviate-client
%pip install tiktoken
%pip install python-dotenv


In [1]:
# Substitua sua chave de API OpenAI:
import openai
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key  = os.environ['OPENAI_API_KEY']
Eddy_WEAVIATE_API_KEY = os.environ["WEAVIATE_API_KEY"]

<h1 align="left"><font color="red">Preparação</font></h1>

NOTA:

Não usei diretamente do meu GITHUB não, porque os dados vem com muitos "campos" adicionais.

In [2]:
import requests
from langchain.document_loaders import TextLoader

#url = "https://github.com/EddyGiusepe/Large_Language_Models_LLMs/blob/main/10_CHROMA_vs_FAISS/macdonalds_euamopontos.txt" #"https://raw.githubusercontent.com/langchain-ai/langchain/master/docs/docs/modules/state_of_the_union.txt"
#res = requests.get(url)


In [3]:
# Status HTTP 200 OK. Esse código indica que a solicitação HTTP foi bem-sucedida!
#res

In [4]:
#with open("./macdonalds_euamopontos.txt", "w") as f:
#    f.write(res.text)


In [5]:
loader = TextLoader('./macdonalds_euamopontos.txt')

documents = loader.load()

In [6]:
documents

[Document(page_content="Neste momento não temos conhecimento de nenhum colaborador infetado com Covid-19.\nNão. A McDonald's não atribui franchisings a sócios ou investidores a qualquer título. O candidato deverá ser capaz de fazer face, pessoalmente, aos requisitos financeiros exigidos e estar disposto a dedicar-se a tempo inteiro à atividade do restaurante McDonald's. Aliás, um franquiado deverá deixar de se dedicar a qualquer outro negócio ou atividade profissional e deverá terminar com sucesso o seu programa de formação e avaliação, que tem a duração média de doze meses.\nEssas placas significam que a energia utilizada nesses restaurantes McDonald's é proveniente, exclusivamente, de fontes renováveis. A chamada 'energia verde', fornecida por uma empresa certificada.\nSim. A McDonald's tem vários projetos e casos na área do ambiente e sustentabilidade que têm como objetivo contribuir para um menor impacto ambiental e garantir as melhores práticas. Por exemplo, implementámos, em 2006

Em seguida, `chunk seus documentos` — Porque o `Document`, em seu estado original, é muito longo para caber na janela de contexto do LLM, você precisa dividi-lo em chunks menores. LangChain vem com muitos divisores de texto ([text splitters](https://python.langchain.com/docs/modules/data_connection/document_transformers/)) integrados para essa finalidade. Para este exemplo simples, você pode usar `CharacterTextSplitter` com um `chunk_size` de cerca de `500` e um `chunk_overlap` de `50` para preservar a continuidade do texto entre os pedaços.

In [7]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter # https://python.langchain.com/docs/modules/data_connection/document_transformers/

#text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,
                                               chunk_overlap=100,
                                               length_function=len,
                                               add_start_index=True,
                                               )


chunks = text_splitter.split_documents(documents)


In [ ]:
chunks

In [9]:
len(chunks)

45

In [10]:
print(chunks[0])
print("")
print(chunks[1])
print("")
print(chunks[2])
print("")
print(chunks[3])
print("")
print(chunks[4])
print("")
print(chunks[5])
print("")
print(chunks[6])
print("")
print(chunks[7])

page_content="Neste momento não temos conhecimento de nenhum colaborador infetado com Covid-19.\nNão. A McDonald's não atribui franchisings a sócios ou investidores a qualquer título. O candidato deverá ser capaz de fazer face, pessoalmente, aos requisitos financeiros exigidos e estar disposto a dedicar-se a tempo inteiro à atividade do restaurante McDonald's. Aliás, um franquiado deverá deixar de se dedicar a qualquer outro negócio ou atividade profissional e deverá terminar com sucesso o seu programa de formação e avaliação, que tem a duração média de doze meses.\nEssas placas significam que a energia utilizada nesses restaurantes McDonald's é proveniente, exclusivamente, de fontes renováveis. A chamada 'energia verde', fornecida por uma empresa certificada." metadata={'source': './macdonalds_euamopontos.txt', 'start_index': 0}

page_content="Sim. A McDonald's tem vários projetos e casos na área do ambiente e sustentabilidade que têm como objetivo contribuir para um menor impacto amb

Por último, `incorpore e armazene os pedaços` — Para ativar a pesquisa semântica nos pedaços de texto, você precisa gerar os `Embeddings vetoriais` para cada pedaço e depois armazene-os junto com seus encaixes. Para gerar os `Embeddings vetoriais`, você pode usar o modelo de `Embedding OpenAI` e, para armazená-los, você pode usar o banco de dados vetorial `Weaviate`. Ao chamar `.from_documents()` o banco de dados vetorial é automaticamente preenchido com os pedaços.

In [11]:
import warnings 
warnings.filterwarnings('ignore') 

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Weaviate
import weaviate
from weaviate.embedded import EmbeddedOptions


client = weaviate.Client(
  embedded_options = EmbeddedOptions(),
  #url="https://eddy-github-rag-q49corr6.weaviate.network", # Replace w/ your endpoint
  #auth_client_secret=weaviate.AuthApiKey(api_key=Eddy_WEAVIATE_API_KEY), # Replace w/ your Weaviate instance API key
)


vectorstore = Weaviate.from_documents(
    client = client,    
    documents = chunks,
    embedding = OpenAIEmbeddings(),
    by_text = False
)


In [13]:
client.is_ready() 

True

<h1 align="left"><font color="yellow">Retrieve (Recuperar)</font></h1>

In [14]:
retriever = vectorstore.as_retriever()

<h1 align="left"><font color="yellow">Augment (Aumentar)</font></h1>

In [15]:
from langchain.prompts import ChatPromptTemplate

template = """Você é um assistente para tarefas de resposta a perguntas.
Use APENAS as seguintes partes do contexto recuperado para responder à pergunta.
Se você não sabe a resposta, basta dizer: "Não sei".
Use no máximo três frases e mantenha a resposta concisa.
Perguntas: {question} 
Contexto: {context} 
Resposta:
"""

prompt = ChatPromptTemplate.from_template(template)

print(prompt)

input_variables=['context', 'question'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='Você é um assistente para tarefas de resposta a perguntas.\nUse APENAS as seguintes partes do contexto recuperado para responder à pergunta.\nSe você não sabe a resposta, basta dizer: "Não sei".\nUse no máximo três frases e mantenha a resposta concisa.\nPerguntas: {question} \nContexto: {context} \nResposta:\n'))]


<h1 align="left"><font color="yellow">Generate (Gerar)</font></h1>

In [20]:
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser


llm = ChatOpenAI(model_name="gpt-3.5-turbo-1106",
                 temperature=0,
                 streaming=True,
                 max_tokens=100,
                 verbose=True
                )


rag_chain = ({"context": retriever,  "question": RunnablePassthrough()} 
            | prompt 
            | llm
            | StrOutputParser() 
        )


In [ ]:
query = "Tem alguém doente com covid19?"

rag_chain.invoke(query)

In [ ]:
query = "Tem delivery no macdonalds"

rag_chain.invoke(query)

In [ ]:
query = "Quem foi ALbert Einstein?"

rag_chain.invoke(query)
